# Exercise 8

In [3]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.morphology import disk
from skimage.measure import label
from skimage.color import label2rgb
from skimage.filters import threshold_otsu

FS = 15 # Fontsize of caption

## 8.1 Grayscale edge detector

In [ ]:
# Load the image in grayscale
I = cv2.imread('bike.png', cv2.IMREAD_GRAYSCALE)

# Normalize the image to 0-1
I = I.astype(np.float32) / 255.0

# Define the structuring element (rectangle with size 4x5)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 4))

# Perform dilation on the original image
I_dilated = cv2.dilate(I, kernel)

# Perform edge detection by subtracting the original image from the dilated image
edge = I_dilated - I

# Threshold to convert to binary image
BW = edge > 0.2  # 0.2 is the threshold value

# Show images
plt.figure(figsize = (10, 10))

plt.subplot(2, 2, 1)
plt.imshow(I, cmap = 'gray')
plt.title('Original Image', fontsize = FS)

plt.subplot(2, 2, 2)
plt.imshow(I_dilated, cmap = 'gray')
plt.title('Dilated Image', fontsize = FS)

plt.subplot(2, 2, 3)
plt.imshow(edge, cmap = 'gray')
plt.title('Edge = Dilation - Origin', fontsize = FS)

plt.subplot(2, 2, 4)
plt.imshow(BW, cmap = 'gray')
plt.title('BW of Edge', fontsize = FS)

plt.tight_layout()
plt.show()


## 8.2 Coins couting using dilation

In [4]:
# Load the input image
I = cv2.imread('coins.png', cv2.IMREAD_GRAYSCALE)

# Threshold the image to create a binary mask
level = threshold_otsu(I)  # Otsu's method for thresholding
BW = 1 - (I > level).astype(np.uint8)  # Convert to binary and invert colors

# a. Without dilation
L = label(BW, connectivity = 2)  # Label connected components
obj = np.max(L)  # Number of objects
rgbLabel1 = label2rgb(L, bg_label = 0, bg_color = (0, 0, 0), colors = 'jet')
print(f"Number of coins before dilation: {obj}")

# b. With dilation
se = disk(25)  # Create a disk-shaped structuring element with radius 25
BW_dilated = 1 - cv2.dilate((I > level).astype(np.uint8), se.astype(np.uint8))

L2 = label(BW_dilated, connectivity=2)  # Label connected components
obj2 = np.max(L2)  # Number of objects after dilation
rgbLabel2 = label2rgb(L2, bg_label = 0, bg_color = (0, 0, 0), colors = 'jet')
print(f"Number of coins after dilation: {obj2}")

# Show the images
plt.figure(figsize=(10, 10))

plt.subplot(2, 2, 1)
plt.imshow(I, cmap='gray')
plt.title('Original Image', fontsize=FS)

plt.subplot(2, 2, 2)
plt.imshow(BW, cmap='gray')
plt.title('Binary Image', fontsize=FS)

plt.subplot(2, 2, 3)
plt.imshow(rgbLabel1)
plt.title('Labeling Before Dilation', fontsize=FS)

plt.subplot(2, 2, 4)
plt.imshow(rgbLabel2)
plt.title('Labeling After Dilation', fontsize=FS)

plt.tight_layout()
plt.show()


KeyError: 'j'